In [4]:
import cv2
import numpy as np

def highlight_color_variations(image1, image2):
    # Read the images
    img1 = cv2.imread(image1)
    img2 = cv2.imread(image2)

    # Resize images to the same dimensions (optional)
    img1 = cv2.resize(img1, (500, 500))
    img2 = cv2.resize(img2, (500, 500))

    # Convert images to Lab color space for better color representation
    lab1 = cv2.cvtColor(img1, cv2.COLOR_BGR2Lab)
    lab2 = cv2.cvtColor(img2, cv2.COLOR_BGR2Lab)

    # Compute the absolute difference between the two images
    diff = cv2.absdiff(lab1, lab2)

    # Split the Lab channels
    l, a, b = cv2.split(diff)

    # Threshold the images to highlight color differences
    threshold = 30
    _, thresh_a = cv2.threshold(a, threshold, 255, cv2.THRESH_BINARY)
    _, thresh_b = cv2.threshold(b, threshold, 255, cv2.THRESH_BINARY)

    # Combine the thresholded images
    color_diff = cv2.bitwise_or(thresh_a, thresh_b)

    # Apply the color difference mask to the original image
    result = cv2.bitwise_and(img1, img1, mask=color_diff)

    # Display the original image and the highlighted color differences
    cv2.imshow("Original Image", img1)
    cv2.imshow("Color Variations Highlighted", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
highlight_color_variations("Shades/shade/1.jpg", "Shades/shade/b.jpg")


In [11]:
import cv2
import numpy as np

def find_red_dots_and_distance(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to the HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for red color in HSV
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])

    # Create a mask to extract red regions
    mask1 = cv2.inRange(hsv, lower_red, upper_red)

    # Define the lower and upper bounds for red color (wraparound in HSV)
    lower_red = np.array([160, 100, 100])
    upper_red = np.array([180, 255, 255])

    # Create a mask to extract red regions
    mask2 = cv2.inRange(hsv, lower_red, upper_red)

    # Combine both masks to get the final mask
    mask = cv2.bitwise_or(mask1, mask2)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours based on their x-coordinate
    contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])

    # Extract the bounding boxes of the contours
    bounding_boxes = [cv2.boundingRect(contour) for contour in contours]

    # Draw lines between opposite pairs of points and calculate horizontal distance
    for i in range(0, len(bounding_boxes)-1, 2):
        x1, y1, w1, h1 = bounding_boxes[i]
        x2, y2, w2, h2 = bounding_boxes[i + 1]

        # Calculate the horizontal distance
        distance = abs(x2 - x1)

        # Draw a line between opposite pairs of points
        cv2.line(image, (x1 + w1 // 2, y1 + h1 // 2), (x2 + w2 // 2, y2 + h2 // 2), (0, 255, 0), 2)

        # Display the distance
        cv2.putText(image, f"{distance} pixels", (min(x1, x2), min(y1, y2) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    image = cv2.resize(image, (400, 400))
    # Display the image with lines and distances
    cv2.imshow("Red Dots and Distances", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
find_red_dots_and_distance("distance3.jpg")


In [14]:
import cv2
import numpy as np

def identify_skipped_stitching(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and improve edge detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Canny edge detector
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area to exclude small details
    min_contour_area = 300
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    # Draw a bounding box around the skipped stitching area
    for contour in filtered_contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the original image with the bounding box
    cv2.imshow("Skipped Stitching Detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
identify_skipped_stitching("skip_fabric.jpg")


In [15]:
import cv2
import numpy as np

def process_video(video_path, output_file, threshold):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the video's frames per second (fps) and width
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    # Create a VideoWriter object for the output video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output_video.avi', fourcc, fps, (width, int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    # Open a text file to write frame numbers and corresponding widths
    with open(output_file, 'w') as text_file:
        frame_number = 0

        # Read the first frame
        ret, frame = cap.read()

        # Process video frames
        while ret:
            # Convert the frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Define a region of interest (ROI) for width measurement (red line in this case)
            roi = gray[200:300, :]

            # Apply edge detection (Canny)
            edges = cv2.Canny(roi, 50, 150)

            # Find contours in the ROI
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Check if contours are found
            if contours:
                # Get the bounding box of the first contour (assuming only one contour for simplicity)
                x, y, w, h = cv2.boundingRect(contours[0])

                # Check if the width is below the threshold
                if w < threshold:
                    # Save frame number and width to the text file
                    text_file.write(f"Frame {frame_number}: Width = {w}\n")

                    # Draw a rectangle around the region of interest
                    cv2.rectangle(frame, (0, 200), (width, 300), (0, 0, 255), 2)

            # Write the frame with the rectangle to the output video
            out.write(frame)

            # Read the next frame
            ret, frame = cap.read()
            frame_number += 1

    # Release video capture and writer objects
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage
video_path = 'video_width.mp4'
output_file = 'output_width_measurements.txt'
threshold = 100  # Set your threshold value here
process_video(video_path, output_file, threshold)


In [17]:
import cv2

def detect_no_ball(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Read the first frame
    ret, frame = cap.read()

    # Define the ROI for front foot detection
    roi = frame[300:600, 400:800]

    # Create background subtractor
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()

    while ret:
        # Apply background subtraction to the ROI
        fg_mask = bg_subtractor.apply(roi)

        # Find contours in the foreground mask
        contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Check if any contours (moving objects) are detected
        if contours:
            # Assuming the first contour corresponds to the bowler's front foot
            x, y, w, h = cv2.boundingRect(contours[0])

            # Check if the front foot is past a certain position (threshold)
            if x + w > 50:  # Adjust the threshold based on your specific scenario
                print("No Ball Detected!")

        # Display the frame
        cv2.imshow("Video", frame)

        # Read the next frame
        ret, frame = cap.read()

        # Break the loop if 'q' is pressed
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    # Release the video capture object
    cap.release()
    cv2.destroyAllWindows()

# Example usage
video_path = 'cricket-2.mp4'
detect_no_ball(video_path)


No Ball Detected!
